In [1]:
from qiskit import *
from qiskit.tools.visualization import *
import random

ベースとなる親クラス

In [2]:
class Person(object):

    def __init__(self):
        self.bn = 10

    # 任意の2進数を生成する。
    def generate_bit(self):
        max = int('1' * self.bn,2)
        tmp = random.randint(1, max)
        return format(tmp,'b').zfill(self.bn)

    # キーを生成する
    def generate_key(self,b1,b2,target):
        key = ''
        x = format(int(b1, 2)^int(b2, 2),'b').zfill(self.bn)
        for i in range(self.bn):
            if '0' == x[i]:
                key += target[i]
        return key

Aliceのクラス

In [3]:
class Alice(Person):

    # 初期化
    def __init__(self):
        super().__init__()

    # Aliceのaを生成する (Aliceの準備 準備1 2つの2進数の生成)
    def generate_a_alice(self):
        self.a_alice = super().generate_bit()

    # Aliceのbを生成する (Aliceの準備 準備1 2つの2進数の生成)
    def generate_b_alice(self):
        self.b_alice = super().generate_bit()

    # Aliceのbを送信する (一致している桁の確認)
    def send_b_alice(self):
        return self.b_alice

    # Aliceが作成した量子状態を送信する make_qbit実行後でないと意味がない(AliceからBobへ量子状態の送信)
    def translate_qbit(self):
        return self.qc_alice,self.q_alice

    # Aliceはa,bをもとに量子状態を生成する (Aliceの準備 準備2 量子状態の生成)
    def make_qbit(self):
        # 量子回路を準備
        self.q_alice = QuantumRegister(self.bn, 'q_alice') 
        self.qc_alice = QuantumCircuit(self.q_alice)
        self.qc_alice = self.__generate_qbit(self.qc_alice,self.q_alice,self.a_alice,self.b_alice,self.bn)
        return self.qc_alice.draw(output='mpl')

    # デモ用
    def show_alice_a_b_for_demo(self):
        print('Alice a : ' + self.a_alice)
        print('Alice b : ' + self.b_alice)

    # デモ用
    def show_alice_b_for_demo(self):
        print('Alice b : ' + self.b_alice)

    # 量子状態を生成する
    def __generate_qbit(self,qci,q,a_alice,b_alice,bn):
        for i in range(bn):
            if '0' == a_alice[bn-1-i]:
                if '1' == b_alice[bn-1-i]:
                    # |+>
                    qci.h(q[i])
            else:
                if '0' == b_alice[bn-1-i]:
                    # |0>
                    qci.x(q[i])
                else:
                    # |->
                    qci.x(q[i])
                    qci.h(q[i])
        qci.barrier(q)
        return qci

Bobのクラス

In [4]:
class Bob(Person):

    # 初期化
    def __init__(self):
        super().__init__()

    # Bobのbを生成する (Bobによる測定 準備 1つの2進数を生成)
    def generate_b_bob(self):
        self.b_bob = super().generate_bit()

    # 量子状態を受信する (AliceからBobへ量子状態の送信)
    def recieve_q_alice(self,q):
        # 量子回路を準備
        self.c_bob = ClassicalRegister(self.bn, 'c_bob')
        self.qc_bob = QuantumCircuit(self.c_bob)
        self.qc_bob = q[0] + self.qc_bob
        self.q_bob = q[1]

    # Bobのbを送信する (一致している桁の確認)
    def send_b_bob(self):
        return self.b_bob

    # Bobによる測定 (Bobによる測定 測定)
    def measure_qbit(self):
        for i in range(self.bn):
            if '0' == self.b_bob[self.bn-1-i]:
                self.__z_measure(self.qc_bob,self.q_bob[i],self.c_bob[i])
            else:
                self.__x_measure(self.qc_bob,self.q_bob[i],self.c_bob[i])
        r = execute(self.qc_bob, Aer.get_backend('qasm_simulator')).result()
        rc = r.get_counts()
        self.result_bob = random.choice(list(rc.keys()))
        return self.result_bob

    # デモ用
    def show_bob_b_for_demo(self):
        print('Bob b   : ' + self.b_bob)

    # Bobによる測定（デモ用）
    def measure_qbit_for_display_circit(self):
        for i in range(self.bn):
            if '0' == self.b_bob[self.bn-1-i]:
                self.__z_measure(self.qc_bob,self.q_bob[i],self.c_bob[i])
            else:
                self.__x_measure(self.qc_bob,self.q_bob[i],self.c_bob[i])

        return self.qc_bob.draw(output='mpl')

    # Bobによる測定（デモ用） measure_qbit_for_displayのあとに実行しないといけない。
    def measure_qbit_for_display_result(self):
        r = execute(self.qc_bob, Aer.get_backend('qasm_simulator')).result()
        rc = r.get_counts()
        self.result_bob = random.choice(list(rc.keys()))
        print("List of results Bob can measure")
        print(rc)
        print("Bob's measurement : " + self.result_bob)
        return plot_histogram(rc)

    # Z測定を行う
    def __z_measure(self,qci,q,c):
        qci.measure(q,c)

    # X測定を行う
    def __x_measure(self,qci,q,c):
        qci.h(q)
        qci.measure(q,c)

アルゴリズムに従ってAliceとBobの通信を疑似的に再現します。

In [5]:
alice = Alice()
bob = Bob()
# Aliceの準備 準備1 2つの2進数の生成
alice.generate_a_alice()
alice.generate_b_alice()
# Aliceの準備 準備2 量子状態の生成
alice.make_qbit()
# Bobによる測定 準備 1つの2進数を生成
bob.generate_b_bob()
# デモ
alice.show_alice_a_b_for_demo()
bob.show_bob_b_for_demo()
# Bobの測定
bob.recieve_q_alice(alice.translate_qbit())
print('Bob result :' + bob.measure_qbit())
# 一致している桁の確認　AliceのbとBobのbを共有する
# 一致している桁の確認　Bob側でキーを生成する
print('Bob key : ' + bob.generate_key(alice.send_b_alice(),bob.b_bob,bob.result_bob))
# 一致している桁の確認　Alice側でキーを生成する
print('Alice key : ' + alice.generate_key(bob.send_b_bob(),alice.b_alice,alice.a_alice) )


Alice a : 0110110001
Alice b : 0100110001
Bob b   : 0001101011
Bob result :0011110011
Bob key : 01101
Alice key : 01101


Eveのクラス(事前漏洩あり)

In [6]:
class Eve(Alice,Bob):

    # 初期化
    def __init__(self,b_alice,b_bob):
        super().__init__()
        self.b_alice = b_alice
        self.b_bob = b_alice
        self.tmp_b_bob = b_bob

    # Eveが測定した値をAliceのaとする
    def set_a_alice(self,a_alice):
        self.a_alice = a_alice

アルゴリズムに従ってAliceとBobの通信をEveが傍受する内容を疑似的に再現します。

In [9]:
alice = Alice()
bob = Bob()
# Aliceの準備 準備1 2つの2進数の生成
alice.generate_a_alice()
alice.generate_b_alice()
# Aliceの準備 準備2 量子状態の生成
alice.make_qbit()
# Bobによる測定 準備 1つの2進数を生成
bob.generate_b_bob()
# デモ
alice.show_alice_a_b_for_demo()
bob.show_bob_b_for_demo()
# Eve登場 AliceのbとBobのbを事前に知っているものとする。(AliceのbとBobのbを間違って先に共有してしまう。)
# Eveの傍受開始
# b,b'傍受
eve = Eve(alice.send_b_alice(),bob.send_b_bob())
# 量子状態傍受
eve.recieve_q_alice(alice.translate_qbit())
# 測定
eve.set_a_alice(eve.measure_qbit())
# 複製
eve.make_qbit()
# Eve側でキーを生成する
print('Eve key : ' + eve.generate_key(eve.tmp_b_bob,eve.b_alice,eve.a_alice))
# Bobの測定
bob.recieve_q_alice(eve.translate_qbit())
bob.measure_qbit()
# Bob側でキーを生成する
print('Bob key : ' + bob.generate_key(alice.send_b_alice(),bob.b_bob,bob.result_bob))
# Alice側でキーを生成する
print('Alice key : ' + alice.generate_key(bob.send_b_bob(),alice.b_alice,alice.a_alice))


Alice a : 1010100110
Alice b : 1111001110
Bob b   : 0100010101
Eve key : 011
Bob key : 011
Alice key : 011


Eveのクラス(事前漏洩なし)

In [10]:
class Eve(Alice,Bob):

    # 初期化
    def __init__(self):
        super().__init__()
        # Eveはaliceのbが分からないので自分で生成しないといけない、ついでにBobのbも
        self.b_alice = super().generate_bit()
        self.b_bob = super().generate_bit()
        self.tmp_b_bob = super().generate_bit()

    # Eveが測定した値をAliceのaとする
    def set_a_alice(self,a_alice):
        self.a_alice = a_alice

アルゴリズムに従ってAliceとBobの通信をEveが傍受する内容を疑似的に再現します。

In [11]:
alice = Alice()
bob = Bob()
# Aliceの準備 準備1 2つの2進数の生成
alice.generate_a_alice()
alice.generate_b_alice()
# Aliceの準備 準備2 量子状態の生成
alice.make_qbit()
# Bobによる測定 準備 1つの2進数を生成
bob.generate_b_bob()
# デモ
alice.show_alice_a_b_for_demo()
bob.show_bob_b_for_demo()
# Eveの傍受開始
eve = Eve()
# 量子状態傍受
eve.recieve_q_alice(alice.translate_qbit())
# 測定
eve.set_a_alice(eve.measure_qbit())
# 複製
eve.make_qbit()
# Bobの測定
bob.recieve_q_alice(eve.translate_qbit())
bob.measure_qbit()
# Bob側でキーを生成する
print('Bob key : ' + bob.generate_key(alice.send_b_alice(),bob.b_bob,bob.result_bob))
# Alice側でキーを生成する
print('Alice key : ' + alice.generate_key(bob.send_b_bob(),alice.b_alice,alice.a_alice))
# Eve側でキーを生成する
print('Eve key : ' + eve.generate_key(eve.tmp_b_bob,eve.b_alice,eve.a_alice))

Alice a : 0011000011
Alice b : 0010010010
Bob b   : 1110111111
Bob key : 1010
Alice key : 1101
Eve key : 000100
